<a href="https://colab.research.google.com/github/aayushs879/Densenet-on-CIFAR-10/blob/master/DNST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
path = os.path.abspath('gdrive/My Drive/Inkers')
#os.mkdir(os.path.join(path, '4thIter'))
path = os.path.join(path, '4thIter')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation, SpatialDropout2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.layers import Concatenate
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 250
l = 12
num_filter = 36 #added 24 more filters
compression = 0.5 
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
# removed the dropout
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        #if dropout_rate>0:
         # Conv2D_3_3 = Dropout2D(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False, kernel_regularizer = regularizers.l1() ,padding='same')(relu)
    #if dropout_rate>0:
      #Conv2D_BottleNeck = Dropout2D(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
# converted the last Dense Layer to a Fully Convolution N/w as use of Dense Layer was prohibited
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    temp = Conv2D(num_classes, kernel_size = (2,2))(AvgPooling)
    output = Activation('softmax')(temp)
    flat = Flatten()(output)
    
    return flat

In [0]:
num_filter = 36
dropout_rate = 0.2
l= 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 32, 32, 36)   972         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 36)   144         conv2d_54[0][0]                  
__________________________________________________________________________________________________
activation_54 (Activation)      (None, 32, 32, 36)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_55 

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)


In [0]:
from keras.callbacks import ModelCheckpoint, CSVLogger
ckpt = ModelCheckpoint(os.path.join(path, 'model.hdf5'), monitor = 'val_acc')
csv = CSVLogger(os.path.join(path, 'log.csv'), append = True)


In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(0.01, momentum = 0.7),
              metrics=['accuracy'])

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '30epochs.h5'))


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/30
391/390 [==============================] - 301s 769ms/step - loss: 7.0762 - acc: 0.4017 - val_loss: 2.7243 - val_acc: 0.2101
Epoch 2/30
391/390 [==============================] - 282s 720ms/step - loss: 1.9663 - acc: 0.4695 - val_loss: 2.2608 - val_acc: 0.3896
Epoch 3/30
391/390 [==============================] - 281s 720ms/step - loss: 1.7373 - acc: 0.5277 - val_loss: 1.9559 - val_acc: 0.4850
Epoch 4/30
391/390 [==============================] - 281s 720ms/step - loss: 1.6122 - acc: 0.5716 - val_loss: 2.2660 - val_acc: 0.4709
Epoch 5/30
391/390 [==============================] - 281s 720ms/step - loss: 1.5173 - acc: 0.5992 - val_loss: 1.9089 - val_acc: 0.5104
Epoch 6/30
391/390 [==============================] - 282s 720ms/step - loss: 1.4472 - acc: 0.6283 - val_loss: 1.7864 - val_acc: 0.5552
Epoch 7/30
391/390 [==============================] - 282s

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '60epochs.h5'))

Epoch 1/30
391/390 [==============================] - 281s 719ms/step - loss: 0.9025 - acc: 0.8154 - val_loss: 0.8638 - val_acc: 0.8250
Epoch 2/30
391/390 [==============================] - 282s 721ms/step - loss: 0.8880 - acc: 0.8205 - val_loss: 1.1500 - val_acc: 0.7579
Epoch 3/30
391/390 [==============================] - 281s 720ms/step - loss: 0.8771 - acc: 0.8245 - val_loss: 1.1167 - val_acc: 0.7605
Epoch 4/30
391/390 [==============================] - 281s 720ms/step - loss: 0.8728 - acc: 0.8261 - val_loss: 1.0960 - val_acc: 0.7643
Epoch 5/30
391/390 [==============================] - 281s 718ms/step - loss: 0.8609 - acc: 0.8320 - val_loss: 1.3143 - val_acc: 0.7280
Epoch 6/30
391/390 [==============================] - 281s 719ms/step - loss: 0.8508 - acc: 0.8326 - val_loss: 1.1231 - val_acc: 0.7702
Epoch 7/30
391/390 [==============================] - 281s 719ms/step - loss: 0.8448 - acc: 0.8333 - val_loss: 0.8784 - val_acc: 0.8293
Epoch 8/30
391/390 [============================

In [0]:
#restoring the last model
from keras.models import load_model
model = load_model(os.path.join(path, 'model.hdf5'))
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '72epochs.h5'))

Epoch 1/30
391/390 [==============================] - 289s 739ms/step - loss: 0.7186 - acc: 0.8692 - val_loss: 0.8482 - val_acc: 0.8320
Epoch 2/30
391/390 [==============================] - 274s 699ms/step - loss: 0.7068 - acc: 0.8725 - val_loss: 0.7481 - val_acc: 0.8645
Epoch 3/30
391/390 [==============================] - 273s 697ms/step - loss: 0.7072 - acc: 0.8747 - val_loss: 0.8450 - val_acc: 0.8378
Epoch 4/30
391/390 [==============================] - 273s 699ms/step - loss: 0.7020 - acc: 0.8749 - val_loss: 1.3341 - val_acc: 0.7237
Epoch 5/30
391/390 [==============================] - 273s 698ms/step - loss: 0.7001 - acc: 0.8760 - val_loss: 0.9239 - val_acc: 0.8147
Epoch 6/30
391/390 [==============================] - 273s 698ms/step - loss: 0.6851 - acc: 0.8795 - val_loss: 0.8432 - val_acc: 0.8421
Epoch 7/30
391/390 [==============================] - 273s 698ms/step - loss: 0.6799 - acc: 0.8812 - val_loss: 0.7917 - val_acc: 0.8431
Epoch 8/30
391/390 [============================

In [0]:
model.load_weights(os.path.join(path, '72epochs.h5'))
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '102epochs.h5'))


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/30
391/390 [==============================] - 293s 749ms/step - loss: 0.6085 - acc: 0.9015 - val_loss: 0.8522 - val_acc: 0.8419
Epoch 2/30
391/390 [==============================] - 276s 706ms/step - loss: 0.6065 - acc: 0.9018 - val_loss: 0.8206 - val_acc: 0.8466
Epoch 3/30
391/390 [==============================] - 274s 701ms/step - loss: 0.6005 - acc: 0.9033 - val_loss: 0.8312 - val_acc: 0.8405
Epoch 4/30
391/390 [==============================] - 274s 702ms/step - loss: 0.5962 - acc: 0.9037 - val_loss: 0.6590 - val_acc: 0.8825
Epoch 5/30
391/390 [==============================] - 274s 700ms/step - loss: 0.5922 - acc: 0.9054 - val_loss: 1.0431 - val_acc: 0.7991
Epoch 6/30
391/390 [==============================] - 273s 699ms/step - loss: 0.5945 - acc: 0.9043 - val_loss: 0.9608 - val_acc: 0.8028
Epoch 7/30
391/390 [==============================] - 273s

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 1.5*x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '132epochs.h5'))

Epoch 1/30
586/585 [==============================] - 404s 689ms/step - loss: 0.5365 - acc: 0.9203 - val_loss: 0.7786 - val_acc: 0.8584
Epoch 2/30
586/585 [==============================] - 403s 688ms/step - loss: 0.5404 - acc: 0.9175 - val_loss: 0.7118 - val_acc: 0.8775
Epoch 3/30
586/585 [==============================] - 402s 686ms/step - loss: 0.5311 - acc: 0.9210 - val_loss: 0.7592 - val_acc: 0.8632
Epoch 4/30
586/585 [==============================] - 402s 686ms/step - loss: 0.5346 - acc: 0.9207 - val_loss: 0.7722 - val_acc: 0.8618
Epoch 5/30
586/585 [==============================] - 402s 686ms/step - loss: 0.5259 - acc: 0.9235 - val_loss: 0.9573 - val_acc: 0.8196
Epoch 6/30
586/585 [==============================] - 402s 686ms/step - loss: 0.5279 - acc: 0.9210 - val_loss: 0.7815 - val_acc: 0.8568
Epoch 7/30
586/585 [==============================] - 402s 687ms/step - loss: 0.5223 - acc: 0.9234 - val_loss: 0.6805 - val_acc: 0.8873
Epoch 8/30
586/585 [============================

In [0]:
keras.backend.set_value(model.optimizer.lr, .001)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3*x_train.shape[0]/batch_size, epochs = 20, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '157epochs.h5'))#157 because it ran for 5 epochs before, as i forgot to update the learning rate

Epoch 1/20
1172/1171 [==============================] - 785s 669ms/step - loss: 0.2054 - acc: 0.9635 - val_loss: 0.3784 - val_acc: 0.9125
Epoch 2/20
1172/1171 [==============================] - 782s 667ms/step - loss: 0.1892 - acc: 0.9646 - val_loss: 0.4052 - val_acc: 0.9070
Epoch 3/20
1172/1171 [==============================] - 783s 668ms/step - loss: 0.1935 - acc: 0.9645 - val_loss: 0.4143 - val_acc: 0.9061
Epoch 4/20
1172/1171 [==============================] - 782s 667ms/step - loss: 0.2025 - acc: 0.9633 - val_loss: 0.4531 - val_acc: 0.9028
Epoch 5/20
1172/1171 [==============================] - 780s 666ms/step - loss: 0.2001 - acc: 0.9644 - val_loss: 0.4063 - val_acc: 0.9105
Epoch 6/20
1172/1171 [==============================] - 779s 665ms/step - loss: 0.1990 - acc: 0.9645 - val_loss: 0.4300 - val_acc: 0.9067
Epoch 7/20
1172/1171 [==============================] - 775s 661ms/step - loss: 0.1998 - acc: 0.9646 - val_loss: 0.4415 - val_acc: 0.9029
Epoch 8/20
1172/1171 [============

In [0]:
model.load_weights(os.path.join(path, '157epochs.h5'))
keras.backend.set_value(model.optimizer.momentum, 0.7)
keras.backend.set_value(model.optimizer.lr, 0.001)
best_ckpt = ModelCheckpoint(os.path.join(path, 'best_model.h5'), monitor = 'val_acc', save_best_only = True)
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3*x_train.shape[0]/batch_size, epochs = 20, validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])
model.save_weights(os.path.join(path, '177epochs.h5'))


Epoch 1/20
1172/1171 [==============================] - 802s 684ms/step - loss: 0.1953 - acc: 0.9678 - val_loss: 0.4215 - val_acc: 0.9101
Epoch 2/20
1172/1171 [==============================] - 802s 685ms/step - loss: 0.1928 - acc: 0.9677 - val_loss: 0.4599 - val_acc: 0.9045
Epoch 3/20
1172/1171 [==============================] - 802s 684ms/step - loss: 0.1898 - acc: 0.9684 - val_loss: 0.4914 - val_acc: 0.8930
Epoch 4/20
1172/1171 [==============================] - 802s 684ms/step - loss: 0.1897 - acc: 0.9688 - val_loss: 0.4312 - val_acc: 0.9123
Epoch 5/20
1172/1171 [==============================] - 801s 683ms/step - loss: 0.1883 - acc: 0.9692 - val_loss: 0.4124 - val_acc: 0.9129
Epoch 6/20
1172/1171 [==============================] - 801s 684ms/step - loss: 0.1909 - acc: 0.9688 - val_loss: 0.4092 - val_acc: 0.9140
Epoch 7/20
1172/1171 [==============================] - 802s 684ms/step - loss: 0.1898 - acc: 0.9688 - val_loss: 0.4962 - val_acc: 0.8972
Epoch 8/20
1172/1171 [============

In [0]:
model.load_weights(os.path.join(path, '177epochs.h5'))
from keras.preprocessing.image import ImageDataGenerator
keras.backend.set_value(model.optimizer.momentum, 0.7)
keras.backend.set_value(model.optimizer.lr, 0.001)
best_ckpt = ModelCheckpoint(os.path.join(path, 'best_model.h5'), monitor = 'val_acc', save_best_only = True)
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3.5*x_train.shape[0]/batch_size, epochs = 10, validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])
model.save_weights(os.path.join(path, '187epochs.h5'))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
1368/1367 [==============================] - 942s 689ms/step - loss: 0.1854 - acc: 0.9706 - val_loss: 0.4150 - val_acc: 0.9147
Epoch 2/10
1368/1367 [==============================] - 926s 677ms/step - loss: 0.1840 - acc: 0.9710 - val_loss: 0.4621 - val_acc: 0.9051
Epoch 3/10
1368/1367 [==============================] - 921s 673ms/step - loss: 0.1822 - acc: 0.9717 - val_loss: 0.4534 - val_acc: 0.9077
Epoch 4/10
1368/1367 [==============================] - 923s 675ms/step - loss: 0.1812 - acc: 0.9724 - val_loss: 0.4615 - val_acc: 0.9039
Epoch 5/10
1368/1367 [==============================] - 922s 674ms/step - loss: 0.1863 - acc: 0.9709 - val_loss: 0.4345 - val_acc: 0.9122
Epoch 6/10
1368/1367 [==============================] - 924s 676ms/step - loss: 0.1828 - acc: 0.9715 - val_loss: 0.4552 - val_acc: 0.9066
Epoch 7/10
1368/1367 [========================

In [0]:
model.load_weights(os.path.join(path, '187epochs.h5'))
datagen = ImageDataGenerator(rotation_range = 10, horizontal_flip = True, width_shift_range = 0.08, height_shift_range = 0.08, zoom_range = 0.15, shear_range = 10)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 4*x_train.shape[0]/batch_size, epochs = 10, validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])
model.save_weights(os.path.join(path, '197epochs.h5'))

Epoch 1/10
1563/1562 [==============================] - 1057s 676ms/step - loss: 0.1200 - acc: 0.9864 - val_loss: 0.4136 - val_acc: 0.9134
Epoch 2/10
1563/1562 [==============================] - 1055s 675ms/step - loss: 0.1280 - acc: 0.9853 - val_loss: 0.4497 - val_acc: 0.9056
Epoch 3/10
1563/1562 [==============================] - 1053s 674ms/step - loss: 0.1266 - acc: 0.9857 - val_loss: 0.4673 - val_acc: 0.9032
Epoch 4/10
1563/1562 [==============================] - 1057s 676ms/step - loss: 0.1277 - acc: 0.9854 - val_loss: 0.4837 - val_acc: 0.8996
Epoch 5/10
1563/1562 [==============================] - 1064s 681ms/step - loss: 0.1270 - acc: 0.9862 - val_loss: 0.4422 - val_acc: 0.9090
Epoch 6/10
1563/1562 [==============================] - 1063s 680ms/step - loss: 0.1260 - acc: 0.9864 - val_loss: 0.4395 - val_acc: 0.9095
Epoch 7/10
1563/1562 [==============================] - 1064s 680ms/step - loss: 0.1272 - acc: 0.9860 - val_loss: 0.5276 - val_acc: 0.8968
Epoch 8/10
1563/1562 [=====

In [0]:
model.load_weights(os.path.join(path, '197epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.0001)
bacth_size = 256
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 5, horizontal_flip = True, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 5)
datagen.fit(x_train)
best_ckpt = ModelCheckpoint(os.path.join(path, 'best_model.h5'), monitor = 'val_acc', save_best_only = True)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3*x_train.shape[0]/batch_size, epochs = 10, validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])
model.save_weights(os.path.join(path, '207epochs.h5'))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
1172/1171 [==============================] - 822s 702ms/step - loss: 0.0498 - acc: 0.9969 - val_loss: 0.3443 - val_acc: 0.9235
Epoch 2/10
1172/1171 [==============================] - 806s 687ms/step - loss: 0.0418 - acc: 0.9972 - val_loss: 0.3436 - val_acc: 0.9238
Epoch 3/10
1172/1171 [==============================] - 805s 687ms/step - loss: 0.0398 - acc: 0.9973 - val_loss: 0.3457 - val_acc: 0.9222
Epoch 4/10
1172/1171 [==============================] - 805s 687ms/step - loss: 0.0384 - acc: 0.9972 - val_loss: 0.3404 - val_acc: 0.9242
Epoch 5/10
1172/1171 [==============================] - 806s 687ms/step - loss: 0.0373 - acc: 0.9972 - val_loss: 0.3404 - val_acc: 0.9234
Epoch 6/10
1172/1171 [==============================] - 806s 687ms/step - loss: 0.0372 - acc: 0.9969 - val_loss: 0.3386 - val_acc: 0.9223
Epoch 7/10
 740/1171 [=================>......

KeyboardInterrupt: ignored

In [0]:
keras.backend.set_value(model.optimizer.lr, 0.0001)
keras.backend.set_value(model.optimizer.momentum, 0.5)
bacth_size = 512
datagen = ImageDataGenerator(rotation_range = 5, horizontal_flip = True, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 5)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3*x_train.shape[0]/batch_size, epochs = 5 validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])

Epoch 1/5
1172/1171 [==============================] - 807s 688ms/step - loss: 0.0342 - acc: 0.9971 - val_loss: 0.3361 - val_acc: 0.9227
Epoch 2/5
1172/1171 [==============================] - 804s 686ms/step - loss: 0.0335 - acc: 0.9973 - val_loss: 0.3415 - val_acc: 0.9216
Epoch 3/5
1172/1171 [==============================] - 804s 686ms/step - loss: 0.0335 - acc: 0.9970 - val_loss: 0.3467 - val_acc: 0.9207
Epoch 4/5
1172/1171 [==============================] - 805s 687ms/step - loss: 0.0338 - acc: 0.9970 - val_loss: 0.3481 - val_acc: 0.9211
Epoch 5/5
1172/1171 [==============================] - 805s 687ms/step - loss: 0.0334 - acc: 0.9969 - val_loss: 0.3397 - val_acc: 0.9218


In [0]:
model.save_weights(os.path.join(path, '208epochs.h5'))

In [0]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 20s 2ms/step


[0.33972805423736574, 0.9218]